# **BERT - Fine-tuning**

<hr>

**Author: Daniel Stancl**

**Description:** The notebook used for fine-tuning BERT implemented by HuggingFace on a down-stream task, sentiment analysis of employee reviews from Glassdoor. This notebook is a part of the MSc project.

Fine-tuning is done using 1 GPU.

### **1. Settings & Import modules**

<hr>

- Install modules necessary for training on TPUs
- Install all non-standard modules
- Import libraries and mount the drive

In [1]:
!pip install pytorch-lightning==0.8.5
!pip install tensorboard==2.3.0
!pip install transformers==2.10.0

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 276kB 61.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=cad9bdafbf5ca77ee0f1131a869446e811f3d89d61609b0305456b5d96c7c9b2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: pytorch-lightning 0.7.6
    Uninstalling pytorch-lightning-0.7.6:
      Successfully uninstalled pytorch-lightning-0.7.6


In [2]:
# import libraries and settings
from os import listdir
from os.path import isfile, join
import numpy as np
from numpy.random import random, seed
import pandas as pd

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
%cd 'drive/My Drive'

Mounted at /content/drive/
/content/drive/My Drive


In [3]:
!nvidia-smi

Tue Aug 11 07:53:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **2. BERT import**

<hr>

- Choose the dir
- Import BERT_SentimentClassifier module
- set parameters for BERT
- instantiate BERT_SentimetnClassifier
- set early stopping rule

In [4]:
# choose the dir
%cd 'MSc Project/'

/content/drive/My Drive/MSc Project


In [5]:
from SentimentClassifier import BERT_SentimentClassifier

In [6]:
BERT_parameters = {
    'PRE_TRAINED_MODEL_NAME': 'bert-base-cased',
    'BATCH_SIZE': 32,
    'NUM_WORKERS': 40,
    'DROPOUT_PROB': 0.3,
    'N_CLASSES': 3,
    'N_EPOCHS': 4,
    'LEARNING_RATE': 4e-5,
    'DATA_PATH': '/content/drive/My Drive/MSc Project/Data/',
}

In [7]:
BERT_clf = BERT_SentimentClassifier(**BERT_parameters)

### **3. BERT training**

<hr>

- Instantiate training with desired parameters
- Run training

In [8]:
# instantiate trainer
trainer = Trainer(
    gpus=1,
    max_epochs=BERT_parameters['N_EPOCHS']
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(BERT_clf)


  | Name    | Type      | Params
--------------------------------------
0 | bert    | BertModel | 108 M 
1 | dropout | Dropout   | 0     
2 | out     | Linear    | 2 K   


Val loss 1.2645,\ Accuracy 0.1094


Val loss 0.9682,\ Accuracy 0.5497


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Train loss 0.9786


### **4. Save fine-tuned model**

In [ ]:
PATH = '/content/drive/My Drive/MSc Project/checkpoint.pt'

torch.save(BERT_clf.state_dict(), PATH)

In [ ]:
model=BERT_SentimentClassifier(**BERT_parameters)
model.load_state_dict(torch.load(PATH))

In [ ]:
BERT_clf